In [12]:
!pip install tqdm

In [15]:
import requests
from tqdm.notebook import tqdm
import os.path
from calendar import monthrange
import time

API_KEY = ""

In [11]:
# Note the months to fetch and generate dates
months = [3, 4]
dates = [f"""2022-{month:02}-{day:02}""" for month in months for day in range(monthrange(2022, month)[1])]

# Show last 10 days to fetch
print(len(dates))
dates[-10:]

61


['2022-04-20',
 '2022-04-21',
 '2022-04-22',
 '2022-04-23',
 '2022-04-24',
 '2022-04-25',
 '2022-04-26',
 '2022-04-27',
 '2022-04-28',
 '2022-04-29']

In [ ]:
# Downloading real-time historical data.
!mkdir -p data/0-koda-gtfs-rt

headers = {'accept': 'application/octet-stream'}

# Loop over all days
def fetch_rt_files(): 
    for date in tqdm(dates):

        # Create parameter dictionary including key for this iteration
        params = {
            'date': date,
            'key': API_KEY,
        }

        # Construct filename and output path.
        # If file is already downloaded, we skip this iteration.
        file_path = f"""./data/0-koda-gtfs-rt/{params['date']}.7z"""
        if os.path.exists(file_path):
            continue;

        # Fetch actual response
        response = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-rt/sl/VehiclePositions', 
                                params=params, headers=headers)

        # If status code is 200 (HTTP OK), save file. 
        if str(response.status_code) == "200":
            with open(file_path, 'wb') as file:
                file.write(response.content)
      
# Execute requests, and wait for a moment to request generated files again.
fetch_rt_files()
time.sleep(60)
fetch_rt_files()

  0%|          | 0/61 [00:00<?, ?it/s]

In [35]:
# Downloading static daily historical data.
!mkdir -p data/0-koda-gtfs-static

headers = {'accept': 'application/octet-stream'}

# Loop over all days
def fetch_static_files(): 
    # Loop over all days from 1 to 26 (last one is not included)
    for date in tqdm(dates):

        # Create parameter dictionary including key for this iteration
        params = {
            'date': date,
            'key': API_KEY,
        }

        # Construct filename and output path.
        # If file is already downloaded, we skip this iteration.
        file_path = f"""./data/0-koda-gtfs-static/{params['date']}.zip"""
        if os.path.exists(file_path):
            continue;

        # Fetch actual response
        response = requests.get('https://api.koda.trafiklab.se/KoDa/api/v2/gtfs-static/sl', 
                                params=params, headers=headers)

        # If status code is 200 (HTTP OK), save file. 
        if str(response.status_code) == "200":
            with open(file_path, 'wb') as file:
                file.write(response.content)


fetch_static_files()
time.sleep(60)
fetch_static_files()

  0%|          | 0/30 [00:00<?, ?it/s]